In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Adrenocortical_Cancer/GSE68950'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"caArray_golub-00327: Sanger cell line Affymetrix gene expression project"
!Series_summary	"The microarray gene expression pattern was studied using 798 different cancer cell lines. The cancer cell lines are obtained from different centers. Annotation information were provided in the supplementary file."
!Series_overall_design	"golub-00327"
!Series_overall_design	"Assay Type: Gene Expression"
!Series_overall_design	"Provider: Affymetrix"
!Series_overall_design	"Array Designs: HT_HG-U133A"
!Series_overall_design	"Organism: Homo sapiens (ncbitax)"
!Series_overall_design	"Tissue Sites: leukemia, Urinary tract, Lung, BiliaryTract, Autonomic Ganglion, Thyroid gland, Stomach, Breast, Pancreas, Head and Neck, Lymphoma, Colorectal, Placenta, Liver, Brain, Bone, pleura, Skin, endometrium, Ovary, cervix, Oesophagus, Connective and Soft Tissue, Muscle, Kidney, Prostate, Adrenal Gland, Eye, Testis, Smooth Muscle Tissue, Vulva, Unknow"
!Series_overall_design	"M

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 1  # disease state: adrenal cortical carcinoma likely under disease state
age_row = None  # Not available from the characteristics data provided
gender_row = None  # Not available from the characteristics data provided

# Function to convert trait (Adrenocortical_Cancer) data
def convert_trait(value):
    if "adrenal cortical carcinoma" in value.lower():
        return 1
    elif "disease state" in value.lower():
        return 0
    else:
        return None

# As age and gender data are not available, we return None from their conversion functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

save_cohort_info('GSE68950', './preprocessed/Adrenocortical_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Adrenocortical_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Adrenocortical_Cancer/trait_data/GSE68950.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM1687570': [0], 'GSM1687571': [0], 'GSM1687572': [0], 'GSM1687573': [0], 'GSM1687574': [0], 'GSM1687575': [0], 'GSM1687576': [0], 'GSM1687577': [0], 'GSM1687578': [0], 'GSM1687579': [0], 'GSM1687580': [0], 'GSM1687581': [0], 'GSM1687582': [0], 'GSM1687583': [0], 'GSM1687584': [0], 'GSM1687585': [0], 'GSM1687586': [0], 'GSM1687587': [0], 'GSM1687588': [0], 'GSM1687589': [0], 'GSM1687590': [0], 'GSM1687591': [0], 'GSM1687592': [0], 'GSM1687593': [0], 'GSM1687594': [0], 'GSM1687595': [0], 'GSM1687596': [0], 'GSM1687597': [0], 'GSM1687598': [0], 'GSM1687599': [0], 'GSM1687600': [0], 'GSM1687601': [0], 'GSM1687602': [0], 'GSM1687603': [0], 'GSM1687604': [0], 'GSM1687605': [0], 'GSM1687606': [0], 'GSM1687607': [0], 'GSM1687608': [0], 'GSM1687609': [0], 'GSM1687610': [0], 'GSM1687611': [0], 'GSM1687612': [0], 'GSM1687613': [0], 'GSM1687614': [0], 'GSM1687615': [0], 'GSM1687616': [0], 'GSM1687617': [0], 'GSM1687618': [0], 'GSM1687619': [0], 'GSM1687620': [0], 'GSM1687621': [0], 'GSM1687622

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': [nan, nan, nan, nan, nan], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens Pax8 mRNA', 'L36861 /FEATURE=expanded_cds /DEFINITION=HUMGCAPB Homo sapiens 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide key names storing identifiers and gene symbols based on the data preview
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Adrenocortical_Cancer/gene_data/GSE68950.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Adrenocortical_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE68950', './preprocessed/Adrenocortical_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Adrenocortical_Cancer/GSE68950.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Adrenocortical_Cancer', the least common label is '1.0' with 2 occurrences. This represents 0.25% of the dataset.
The distribution of the feature 'Adrenocortical_Cancer' in this dataset is severely biased.

